In [1]:
from sklearn import datasets
from sklearn.model_selection import cross_val_score

In [27]:
X = datasets.load_boston()['data']
y = datasets.load_boston()['target']

In [24]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import make_scorer
import numpy as np
def sk_cv(model_nm, params, X_train, y_train, cv):

    model_nm = model_nm.lower()
    if model_nm == 'rf':
        sk_model = RandomForestRegressor(**params)
    elif model_nm == 'et':
        sk_model = ExtraTreesRegressor(**params)
    else:
        raise ValueError(f"model_nm: {model_nm} is not supported now! ")
    def calc_rmse(y, y_pred):
        return np.sqrt(np.mean((y - y_pred)**2))
    # Creating root mean square error for sklearns crossvalidation
    rmse_scorer = make_scorer(calc_rmse, greater_is_better=False)
    
    scores = cross_val_score(sk_model,
                             X_train,
                             y_train,
                             cv=cv,
                             n_jobs=-1,
                             pre_dispatch=10,
                             verbose=0,
                             scoring='neg_mean_absolute_error')
    return scores


params = {'n_estimators':1,'random_state': 2019}
sk_cv('rf',params,X,y,5).mean()

-37.68741758881771

In [28]:
X

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [45]:
import lightgbm as lgb
import xgboost as xgb

params = {'n_estimators':100,'random_state': 2019, 'eval_metric': 'mae'}


# train_set = lgb.Dataset(X,y)
train_set = xgb.DMatrix(X, y)

# cv_dict = lgb.cv(params,
#                  train_set,
#                  num_boost_round=5000,
#                  folds=None,
#                  nfold=5,
#                  stratified=False,
#                  shuffle=False,
#                  metrics=None,
#                  fobj=None,
#                  feval=None,
#                  init_model=None,
#                  feature_name='auto',
#                  categorical_feature='auto',
#                  early_stopping_rounds=100,
#                  fpreproc=None,
#                  verbose_eval=10,
#                  show_stdv=True,
#                  seed=0,
#                  callbacks=None)

cv_dict = xgb.cv(params,
                 train_set,
                 num_boost_round=5000,
                 nfold=5,
                 stratified=False,
                 folds=None,
                 early_stopping_rounds=100,
                 as_pandas=False,
                 verbose_eval=10,
                 seed=0,
                 shuffle=False)

[0]	train-mae:15.5735+0.793929	test-mae:15.7421+3.51174
[10]	train-mae:1.06014+0.0254148	test-mae:2.99867+0.489694
[20]	train-mae:0.53751+0.034997	test-mae:2.95754+0.43464
[30]	train-mae:0.315453+0.01641	test-mae:2.96762+0.432933
[40]	train-mae:0.186519+0.00670086	test-mae:2.97895+0.436536
[50]	train-mae:0.117939+0.00750274	test-mae:2.97607+0.432933
[60]	train-mae:0.074901+0.00447001	test-mae:2.97781+0.435716
[70]	train-mae:0.0488766+0.00327568	test-mae:2.97789+0.437246
[80]	train-mae:0.0308682+0.00229479	test-mae:2.97935+0.436738
[90]	train-mae:0.0208236+0.00194361	test-mae:2.97959+0.43643
[100]	train-mae:0.0134086+0.00086214	test-mae:2.97979+0.435725
[110]	train-mae:0.0087322+0.000696596	test-mae:2.97983+0.436107


In [46]:
cv_dict

{'train-mae-mean': [15.573524,
  11.0279256,
  7.824921999999999,
  5.5841146,
  4.018898,
  2.9415148,
  2.2218834000000003,
  1.7468472000000002,
  1.419397,
  1.201376,
  1.0601448,
  0.9532958,
  0.8801132,
  0.8118550000000001,
  0.7644474000000001],
 'train-mae-std': [0.7939290806065737,
  0.5523860095746818,
  0.3854034031562253,
  0.2598893603505922,
  0.17205939220164637,
  0.11927926296117024,
  0.07883110865540317,
  0.052145477849570056,
  0.037658990108604906,
  0.02616324821577017,
  0.025414789295998483,
  0.0320198516761087,
  0.03568648158280666,
  0.04025582535236359,
  0.042749986791109085],
 'test-mae-mean': [15.7421038,
  11.2778956,
  8.265155799999999,
  6.2800256,
  4.9559526,
  4.1483114,
  3.6077269999999997,
  3.3057122,
  3.154813,
  3.0596796,
  2.9986692,
  2.9730026,
  2.9752308000000003,
  2.9487932,
  2.9485806000000006],
 'test-mae-std': [3.5117384628196557,
  2.821401414466903,
  2.476121886254988,
  2.0730959679101786,
  1.6259049287904384,
  1.20267